# Next Word Predicition - NLP

Mounting the Colab Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq
import urllib.request as url
from bs4 import BeautifulSoup as soup

Using TensorFlow backend.


# Extracting text from Wikipedia

In [3]:
path = 'https://en.wikipedia.org/wiki/Machine_learning'

RText = url.urlopen(path)
div = RText.read()
div = soup(div,'html.parser')
print(div.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Machine learning - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1e366feb-6f78-4e31-93f5-697aff7ecc74","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Machine_learning","wgTitle":"Machine learning","wgCurRevisionId":957340896,"wgRevisionId":957340896,"wgArticleId":233488,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: uses authors parameter","Articles with short description","Articles with long short description","Commons category link from Wikidat

In [4]:
# - finding all the text within the paragraph tag
TextList = []
TextDiv = div.findAll("p")
for i in TextDiv:
  text = i.getText()
  TextList.append(text)
print(TextList)

['Machine learning (ML) is the study of computer algorithms that improve automatically through experience.[1] It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.[2][3]:2 Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop  conventional algorithms to perform the needed tasks.\n', 'Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning.[4][5] In its application across business problems, machine learning

In [5]:
import re

TextWiki = []

for i in TextList:
  text = i.split(" ")
  TextList = []
  for j in text:
    j = j.replace("."," ") # - 
    j = j.replace("\n"," ")
    j = re.sub('[0-9]', '', j) # --- Remove all numeric characters
    j = re.sub('[^a-zA-Z0-9\n\.]', '', j) # --- Remove all special characters
    TextList.append(j)
  TextWiki.extend(TextList)
print(TextWiki)

['Machine', 'learning', 'ML', 'is', 'the', 'study', 'of', 'computer', 'algorithms', 'that', 'improve', 'automatically', 'through', 'experience', 'It', 'is', 'seen', 'as', 'a', 'subset', 'of', 'artificial', 'intelligence', 'Machine', 'learning', 'algorithms', 'build', 'a', 'mathematical', 'model', 'based', 'on', 'sample', 'data', 'known', 'as', 'training', 'data', 'in', 'order', 'to', 'make', 'predictions', 'or', 'decisions', 'without', 'being', 'explicitly', 'programmed', 'to', 'do', 'so', 'Machine', 'learning', 'algorithms', 'are', 'used', 'in', 'a', 'wide', 'variety', 'of', 'applications', 'such', 'as', 'email', 'filtering', 'and', 'computer', 'vision', 'where', 'it', 'is', 'difficult', 'or', 'infeasible', 'to', 'develop', '', 'conventional', 'algorithms', 'to', 'perform', 'the', 'needed', 'tasks', 'Machine', 'learning', 'is', 'closely', 'related', 'to', 'computational', 'statistics', 'which', 'focuses', 'on', 'making', 'predictions', 'using', 'computers', 'The', 'study', 'of', 'math

In [9]:
print(len(TextWiki))

6070


In [0]:
# - https://medium.com/analytics-vidhya/build-a-simple-predictive-keyboard-using-python-and-keras-b78d3c88cffb

unique_TextWiki = np.unique(TextWiki)
unique_TextWiki_index = dict((c, i) for i, c in enumerate(unique_TextWiki))

In [8]:
print(unique_TextWiki_index)

{'': 0, 'A': 1, 'AI': 2, 'AICS': 3, 'AIIts': 4, 'ANN': 5, 'ANNs': 6, 'ATT': 7, 'AUC': 8, 'According': 9, 'Active': 10, 'Adaptive': 11, 'After': 12, 'Agrawal': 13, 'Alan': 14, 'Although': 15, 'American': 16, 'An': 17, 'Anomalies': 18, 'Area': 19, 'Array': 20, 'Arthur': 21, 'Artificial': 22, 'Arun': 23, 'As': 24, 'Association': 25, 'Attempts': 26, 'Based': 27, 'Bayesian': 28, 'Because': 29, 'Big': 30, 'Breiman': 31, 'But': 32, 'By': 33, 'CAA': 34, 'Can': 35, 'Chaos': 36, 'Characteristic': 37, 'Cinematch': 38, 'Classic': 39, 'Classification': 40, 'Cluster': 41, 'Computing': 42, 'Concern': 43, 'Conversely': 44, 'Crossbar': 45, 'Curve': 46, 'DAG': 47, 'Daniel': 48, 'Data': 49, 'Decision': 50, 'Deep': 51, 'Different': 52, 'Dr': 53, 'Duda': 54, 'Due': 55, 'E': 56, 'ECML': 57, 'Each': 58, 'Early': 59, 'Efficient': 60, 'Ehud': 61, 'Evaluated': 62, 'Examples': 63, 'Excel': 64, 'FNR': 65, 'FPR': 66, 'False': 67, 'Feature': 68, 'Federated': 69, 'FeiFei': 70, 'For': 71, 'GA': 72, 'Gboard': 73, 'Gen

In [13]:
WORD_LENGTH = 4
prev_words = []
next_words = []
for i in range(len(TextWiki) - WORD_LENGTH):
    prev_words.append(TextWiki[i:i + WORD_LENGTH])
    next_words.append(TextWiki[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['Machine', 'learning', 'ML', 'is']
the


In [0]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_TextWiki)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_TextWiki)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_TextWiki_index[each_word]] = 1
    Y[i, unique_TextWiki_index[next_words[i]]] = 1

In [18]:
print(X[0][0])

[False False False ... False False False]


# Building the LSTM model

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_TextWiki))))
model.add(Dense(len(unique_TextWiki)))
model.add(Activation('softmax'))

In [21]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Train on 5762 samples, validate on 304 samples
Epoch 1/20
5762/5762 [==============================] - 1s 223us/step - loss: 5.5682 - accuracy: 0.1295 - val_loss: 7.4810 - val_accuracy: 0.0757
Epoch 2/20
5762/5762 [==============================] - 1s 158us/step - loss: 4.7405 - accuracy: 0.1939 - val_loss: 7.9741 - val_accuracy: 0.0493
Epoch 3/20
5762/5762 [==============================] - 1s 158us/step - loss: 3.9290 - accuracy: 0.2747 - val_loss: 8.1033 - val_accuracy: 0.0493
Epoch 4/20
5762/5762 [==============================] - 1s 154us/step - loss: 3.0983 - accuracy: 0.3898 - val_loss: 9.4851 - val_accuracy: 0.0559
Epoch 5/20
5762/5762 [==============================] - 1s 159us/step - loss: 2.3636 - accuracy: 0.5196 - val_loss: 8.9972 - val_accuracy: 0.0461
Epoch 6/20
5762/5762 [==============================] - 1s 156us/step - loss: 1.7264 - accuracy: 0.6623 - val_loss: 11.3495 - val_accuracy: 0.0395
Epoch 7/20
5762/5762 [==============================] - 1s 155us/step - loss

In [0]:
# sm_pickle_X_test = '/content/gdrive/My Drive/Journal Paper/Consumer Compliants/sm_X_test_tfidf.pkl'

model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [0]:
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [25]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_TextWiki)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_TextWiki_index[word]] = 1
    return x
prepare_input("It is not a".lower())

it
is
not
a


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [0]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [0]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_TextWiki[idx] for idx in next_indices]

In [34]:
tokenizer = RegexpTokenizer(r'\w+')
q =  "The study of mathematical optimization delivers"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:4])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  The study of mathematical optimization delivers
Sequence:  the study of mathematical
the
study
of
mathematical
next possible words:  ['algorithms', 'that', 'model', 'and', 'learning']
